In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from numpy import *
import matplotlib.pyplot as plt
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/mlproject2/Training Data.csv')
target_data = pd.read_csv('/content/drive/MyDrive/mlproject2/Training Data Target.csv')

In [ ]:
data

,user_id,aov,category
0,37327,29128.0,Phones
1,37327,354.0,Fashion
2,37327,460.0,Home Decor
3,37327,761.0,Fashion
4,41260,16658.0,Phones
...,...,...,...
257402,29237,689.0,Fashion
257403,15197,2195.0,Groceries
257404,2114,559.0,Back to School
257405,35741,18451.0,Phones


In [ ]:
del data["aov"]

In [ ]:
data["user_id"].value_counts().describe()

count    29972.000000
mean         8.588249
std          3.376469
min          1.000000
25%          6.000000
50%          9.000000
75%         11.000000
max         26.000000
Name: user_id, dtype: float64

In [ ]:
category = ['Phones', 'Fashion', 'Home Decor', 'Toys','Groceries','Back to School','Gaming','Consumer Durables','Books','Kitchen cleaning Supplies','Beauty Products','Ereaders','Board Games','Painiting Supplies','Pet Supplies','Fitness','Laptops','TVs']
category_id = list(range(1,19))
# get the list of tuples from two lists.
# and merge them by using zip().
list_of_tuples = list(zip(category, category_id))
df = pd.DataFrame(list_of_tuples,columns = ['category', 'category_id'])
merged_df = pd.merge(data,df,on='category')
del merged_df["category"]
merged_df

,user_id,category_id
0,37327,1
1,41260,1
2,19360,1
3,19360,1
4,19360,1
...,...,...
257402,46282,10
257403,47716,10
257404,11175,10
257405,20009,10


In [ ]:
merged_df.insert(2, "freq", list(ones(data.shape[0])), True)
merged_df

,user_id,category_id,freq
0,37327,1,1.0
1,41260,1,1.0
2,19360,1,1.0
3,19360,1,1.0
4,19360,1,1.0
...,...,...,...
257402,46282,10,1.0
257403,47716,10,1.0
257404,11175,10,1.0
257405,20009,10,1.0


In [ ]:
freqs = merged_df.groupby(['user_id','category_id'])['freq'].count()
dataset = freqs.to_frame()

In [ ]:
dataset.reset_index(inplace=True)
# del table['index']
dataset = dataset.rename(columns={"user_id": "userID", "category_id": "itemID", "freq":"rating"})
dataset

,userID,itemID,rating
0,1,1,1
1,1,2,2
2,1,4,1
3,1,5,1
4,1,16,2
...,...,...,...
159864,50075,1,1
159865,50078,9,1
159866,50079,10,1
159867,50080,2,1


In [ ]:
dataset['rating'].describe()

count    159869.000000
mean          1.610112
std           0.944003
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max           9.000000
Name: rating, dtype: float64

In [ ]:
pip install scikit-surprise

In [ ]:
table = dataset.pivot_table(values='rating', index='userID',columns='itemID')
table

itemID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
userID,,,,,,,,,,,,,,,,,,
1,1.0,2.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,3.0,2.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,NaN,3.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3.0,4.0,NaN,2.0,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
7,2.0,2.0,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50075,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import surprise
from surprise import Reader, Dataset, SVD, BaselineOnly, KNNBasic, KNNWithMeans, KNNBaseline, KNNWithZScore, SlopeOne
from surprise.model_selection import cross_validate

## Testing Trained model on Target_predictions

In [ ]:
del target_data['aov']
target_data

,user_id,category
0,13153,Phones
1,42853,Phones
2,30550,Phones
3,9797,Phones
4,8171,Phones
...,...,...
13240,49383,Fashion
13241,49590,Home Decor
13242,49799,Fitness
13243,49900,Fitness


In [ ]:
target_data["category"].value_counts()

Phones                       7124
Fashion                      2195
Pet Supplies                  910
Ereaders                      887
Fitness                       874
TVs                           426
Laptops                       308
Home Decor                    238
Consumer Durables             186
Toys                           58
Back to School                 19
Gaming                         10
Groceries                       8
Kitchen cleaning Supplies       2
Name: category, dtype: int64

In [ ]:
category = ['Phones', 'Fashion', 'Home Decor', 'Toys','Groceries','Back to School','Gaming','Consumer Durables','Books','Kitchen cleaning Supplies','Beauty Products','Ereaders','Board Games','Painiting Supplies','Pet Supplies','Fitness','Laptops','TVs']

item_dict = {}
for i in range(1,19):
  item_dict[i] = category[i-1]

In [ ]:
#@title Default title text
# top1 = [[] for i in range(len(target_data))] 
# accuracy = 0
# from operator import itemgetter
# # Cross-validation of parameters
# for i in range(len(target_data)):
#     for iid in range(1,19):
#         est = algo1.predict(target_data.iloc[i][0], iid, verbose=False).est
#         top1[i].append([iid,est])
#     b=sorted(top1[i], key=itemgetter(1),reverse=True)
#     if (item_dict[b[0][0]] == target_data.iloc[i].iloc[1]) : 
#       accuracy=accuracy+1
  
# accuracy = accuracy/(len(target_data))
# accuracy

In [ ]:
#@title
# table = merged_df.pivot_table(values='count', index='user_id',columns='category_id', aggfunc=sum)
# cross_validate(algo1, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
#@title
# import matplotlib.pyplot as plt
# import seaborn as sns
# df = data.corr()
# sns.heatmap(df,annot = True,square = True)
# plt.show()

## Processing test data

In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/mlproject2/Test Data.csv')
del test_data["Unnamed: 0"]

In [ ]:
del test_data["aov"]

In [ ]:
test_data["category"].value_counts()

Phones                       44627
Fashion                      40082
Home Decor                   19911
Toys                         14096
Groceries                     8559
Back to School                8405
Gaming                        8109
Consumer Durables             7984
Books                         3968
Ereaders                      2163
Pet Supplies                  2059
Beauty Products               2055
Kitchen cleaning Supplies     2042
Painiting Supplies            1987
Board Games                   1981
Fitness                       1961
Laptops                       1310
TVs                           1255
Name: category, dtype: int64

In [ ]:
test_data["user_id"].value_counts().describe()

count    19981.000000
mean         8.635904
std          3.366444
min          1.000000
25%          6.000000
50%          9.000000
75%         11.000000
max         26.000000
Name: user_id, dtype: float64

In [ ]:
category = ['Phones', 'Fashion', 'Home Decor', 'Toys','Groceries','Back to School','Gaming','Consumer Durables','Books','Kitchen cleaning Supplies','Beauty Products','Ereaders','Board Games','Painiting Supplies','Pet Supplies','Fitness','Laptops','TVs']
category_id = list(range(1,19))
# get the list of tuples from two lists.
# and merge them by using zip().
list_of_tuples = list(zip(category, category_id))
df = pd.DataFrame(list_of_tuples,columns = ['category', 'category_id'])
merged_df = pd.merge(test_data,df,on='category')
del merged_df["category"]
merged_df

,user_id,category_id
0,43323,1
1,43323,1
2,43323,1
3,43323,1
4,4269,1
...,...,...
172549,459,14
172550,3215,14
172551,927,14
172552,927,14


In [ ]:
test_data

,user_id,category
0,43323,Phones
1,43323,Phones
2,43323,Phones
3,43323,Phones
4,43323,Fashion
...,...,...
172549,12403,Groceries
172550,31036,Back to School
172551,12652,Fashion
172552,35726,Kitchen cleaning Supplies


In [ ]:
merged_df.insert(2, "freq", list(ones(test_data.shape[0])), True)
merged_df

,user_id,category_id,freq
0,43323,1,1.0
1,43323,1,1.0
2,43323,1,1.0
3,43323,1,1.0
4,4269,1,1.0
...,...,...,...
172549,459,14,1.0
172550,3215,14,1.0
172551,927,14,1.0
172552,927,14,1.0


In [ ]:
freqs = merged_df.groupby(['user_id','category_id'])['freq'].count()
test_dataset = freqs.to_frame()

In [ ]:
test_dataset.reset_index(inplace=True)
test_dataset = test_dataset.rename(columns={"user_id": "userID", "category_id": "itemID", "freq":"rating"})
test_dataset

,userID,itemID,rating
0,2,1,2
1,2,2,2
2,2,12,1
3,2,16,1
4,3,1,3
...,...,...,...
107058,50037,5,1
107059,50050,5,1
107060,50050,6,1
107061,50074,10,1


In [ ]:
test_dataset['rating'].describe()

count    107063.000000
mean          1.611705
std           0.941935
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max           8.000000
Name: rating, dtype: float64

In [ ]:
frames = [dataset, test_dataset]
train_dataset = pd.concat(frames,ignore_index=True)
train_dataset

,userID,itemID,rating
0,1,1,1
1,1,2,2
2,1,4,1
3,1,5,1
4,1,16,2
...,...,...,...
266927,50037,5,1
266928,50050,5,1
266929,50050,6,1
266930,50074,10,1


In [ ]:
user_interactions = {}
for user_id in train_dataset["userID"].unique():
  user_interactions[user_id] = 0
for i in range(len(train_dataset)):
  user_interactions[train_dataset.iloc[i].iloc[0]] = user_interactions[train_dataset.iloc[i].iloc[0]] + train_dataset.iloc[i].iloc[2]

In [ ]:
new_train_dataset = train_dataset
remove= []
for i in range(len(new_train_dataset)):
  if user_interactions[new_train_dataset.iloc[i][0]]<=5:
    remove.append(i)

In [ ]:
new_train_dataset  = new_train_dataset.drop(new_train_dataset .index[remove])
new_train_dataset

In [ ]:
user_interactions[19360]

In [ ]:
target_data

In [ ]:
# A reader is still needed but only the rating_scale param is required.
reader = Reader(rating_scale=(1, 12))
# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(new_train_dataset[['userID', 'itemID', 'rating']], reader)
trainset = data.build_full_trainset()

In [ ]:
for nfactors in [4]:
  for nepochs in [200]:
    for lrall in [5e-4]:
      for regall in [2]:
              # Build an algorithm, and train it.
              algo0 = SVD(n_factors = nfactors, n_epochs=nepochs, lr_all = lrall ,reg_all = regall, biased=True, random_state=9)
              algo0.fit(trainset)

              top1 = [[] for i in range(len(target_data))] 
              accuracy = 0
              from operator import itemgetter
              # Cross-validation of parameters
              for i in range(len(target_data)):
                  for iid in range(1,19):
                      est = algo0.predict(target_data.iloc[i][0], iid, verbose=False).est
                      top1[i].append([iid,est])
                  b=sorted(top1[i], key=itemgetter(1),reverse=True)
                  if (item_dict[b[0][0]] == target_data.iloc[i].iloc[1]) : 
                    accuracy=accuracy+1
              accuracy = accuracy/(len(target_data))
              print("For n_factors = {},n_epochs = {},lr_all = {},reg_all = {} accuracy is: {}".format(nfactors,nepochs,lrall,regall,accuracy) )

In [ ]:
target_data.iloc[i].iloc[1]

In [ ]:
bsl_options = {'method': 'als',
               'n_epochs': 10,
               'reg_u': 5,
               'reg_i': 10
               }
algo2 = BaselineOnly(bsl_options=bsl_options)
algo2.fit(trainset)

In [ ]:
sim_options = {'name': 'pearson_baseline',
               'user_based': False  # compute  similarities between items
               }
algo3 = KNNBasic(sim_options=sim_options)
algo3.fit(trainset)

In [ ]:
top1 = [[] for i in range(len(target_data))] 
accuracy = 0
from operator import itemgetter
# Cross-validation of parameters
for i in range(len(target_data)):
   if user_interactions[target_data.iloc[i][0]]>7:
      for iid in range(1,19):
          est = algo0.predict(target_data.iloc[i][0], iid, verbose=False).est 
          top1[i].append([iid,est])
      b=sorted(top1[i], key=itemgetter(1),reverse=True)
      if (item_dict[b[0][0]] == target_data.iloc[i].iloc[1]) : 
        accuracy=accuracy+1
   else: 
     for iid in range(1,19):
          est = algo2.predict(target_data.iloc[i][0], iid, verbose=False).est
          top1[i].append([iid,est])
     b=sorted(top1[i], key=itemgetter(1),reverse=True)
     if ('Phones' == target_data.iloc[i].iloc[1]) : 
        accuracy=accuracy+1
        
accuracy = accuracy/(len(target_data))
accuracy 

##Creating csv for submission

In [ ]:
reader = Reader(rating_scale=(1, 10))

# The columns must correspond to user id, item id and ratings (in that order).
data2 = Dataset.load_from_df(new_train_dataset[['userID', 'itemID', 'rating']], reader)
trainset1 = data2.build_full_trainset()

# Build an algorithm, and train it.
algo = SVD(n_factors = 4, n_epochs=170, biased=True, lr_all = 5e-4 ,reg_all = 0.1, random_state=9)
algo.fit(trainset1)

In [ ]:
data1 = pd.read_csv("/content/drive/MyDrive/mlproject2/Sample Submission.csv")
data1

In [ ]:
data1

In [ ]:
from operator import itemgetter
b=sorted(top3[0], key=itemgetter(1),reverse=True)
b
category = ['Phones', 'Fashion', 'Home Decor', 'Toys','Groceries','Back to School','Gaming','Consumer Durables','Books','Kitchen cleaning Supplies','Beauty Products','Ereaders','Board Games','Painiting Supplies','Pet Supplies','Fitness','Laptops','TVs']


In [ ]:
del data1['pred3']

In [ ]:
category = ['Phones', 'Fashion', 'Home Decor', 'Toys','Groceries','Back to School','Gaming','Consumer Durables','Books','Kitchen cleaning Supplies','Beauty Products','Ereaders','Board Games','Painiting Supplies','Pet Supplies','Fitness','Laptops','TVs']
top = [[] for i in range(len(data1))]  
item_dict = {}
for i in range(1,19):
  item_dict[i] = category[i-1]
  
pred=zeros((len(data1), 3),dtype = object)
for i in range(len(data1)):
  if user_interactions[data1.iloc[i][0]]>5:
      for iid in range(1,19):
          est = algo0.predict(data1.iloc[i][0], iid, verbose=False).est 
          top[i].append([iid,est])
      b=sorted(top[i], key=itemgetter(1),reverse=True)
      pred[i][0]=item_dict[b[0][0]]
      pred[i][1]=item_dict[b[1][0]]
      pred[i][2]=item_dict[b[2][0]]
  else:
      pred[i][0]='Phones'
      pred[i][1]='Fashion'
      pred[i][2] ='Home Decor'

pred3 = zeros((len(data1)),dtype = object)
for i in range(len(data1)):
  pred3[i] = pred[i][0] + ", " + pred[i][1] + ", " + pred[i][2]


In [ ]:
data1['pred3'] = pred3

In [ ]:
# My final Submission 
h=['user_id','pred3']
data1.to_csv("recomm4.csv",header=h,index=False)